# using undocumented api from lolesports.com


In [ ]:
import requests
import json
from urllib.parse import urlencode
from datetime import datetime, timedelta


def query_lol_esports_api(
    start_date, end_date, language="en-US", event_states=["completed"], page_size=100
):
    """
    Query the LoL Esports API for events within a date range using a POST request.

    Args:
        start_date (str): Start date in ISO 8601 format (e.g., "2025-03-08T23:00:00.000Z")
        end_date (str): End date in ISO 8601 format (e.g., "2025-03-10T22:59:59.999Z")
        language (str): Language code (e.g., "en-US")
        event_states (list): List of event states to filter (e.g., ["completed"])
        page_size (int): Number of results to return

    Returns:
        dict: JSON response from the API, or None if the request fails
    """
    # Base URL for the API
    base_url = "https://lolesports.com/api/gql"

    # Define the variables for the GraphQL query
    variables = {
        "hl": language,
        "sport": "lol",
        "eventDateStart": start_date,
        "eventDateEnd": end_date,
        "eventState": event_states,
        "eventType": "all",
        "vodType": ["recap"],
        "pageSize": page_size,
    }

    # Extensions for the persisted query (fixed for the 'homeEvents' operation)
    extensions = {
        "persistedQuery": {
            "version": 1,
            "sha256Hash": "089916a64423fe9796f6e81b30e9bda7e329366a5b06029748c610a8e486d23f",
        }
    }

    # Request payload
    payload = {
        "operationName": "homeEvents",
        "variables": variables,
        "extensions": extensions,
    }

    # Headers based on the website request
    headers = {
        "Content-Type": "application/json",
        "Accept": "*/*",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8,fr;q=0.7,zh-CN;q=0.6,zh;q=0.5",  # Optional, reflects browser preference
        "apollographql-client-name": "Esports Web",  # Required to identify the client
        "apollographql-client-version": "ab5e5e7",  # Required to identify the client version
        "Referer": "https://lolesports.com/en-US",  # Matches the website context
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",  # Mimics the browser
    }

    try:
        # Make the HTTP POST request
        response = requests.post(base_url, json=payload, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error querying the API: {e}")
        print(
            f"Response text: {response.text if 'response' in locals() else 'No response'}"
        )
        return None

In [ ]:
def get_completed_game_ids(api_response):
    """
    Extract game IDs from completed games in the API response.
    
    Args:
        api_response (dict): JSON response from the LoL Esports API
    
    Returns:
        list: List of game IDs for completed games
    """
    game_ids = []
    
    if not api_response or "data" not in api_response:
        print("Invalid API response or no data found.")
        return game_ids
    
    # Navigate to the events array
    events = api_response.get("data", {}).get("esports", {}).get("events", [])
    
    for event in events:
        # Ensure the event has a match
        match = event.get("match", {})
        if not match:
            continue
            
        # Get the games array from the match
        games = match.get("games", [])
        
        # Extract IDs from completed games
        for game in games:
            if game.get("state") == "completed":
                game_id = game.get("id")
                if game_id:
                    game_ids.append(game_id)
    
    return game_ids

In [ ]:
start_date = "2025-03-08T23:00:00.000Z"
end_date = "2025-03-09T22:59:59.999Z"

# Query the API
response = query_lol_esports_api(
    start_date=start_date,
    end_date=end_date,
    language="en-US",
    event_states=["completed"],
    page_size=100,
)

if response:
    # Extract game IDs
    game_ids = get_completed_game_ids(response)

    # Print the results
    if game_ids:
        print("Completed Game IDs:")
        for game_id in game_ids:
            print(game_id)
    else:
        print("No completed games found.")

In [ ]:
game_id = game_ids[0]
game_id